# Sistemas Inteligentes

## Curso académico 2024-2025

### Laboratorio 2: Búsqueda Metaheurística

#### Instructores

* Juan Carlos Alfaro Jiménez: JuanCarlos.Alfaro@uclm.es
* María Julia Flores Gallego: Julia.Flores@uclm.es
* Ismael García Varea: Ismael.Garcia@uclm.es
* Adrián Rodríguez López: Adrian.Rodriguez18@alu.uclm.es

#### Alumnos Grupo 1
* Ana Barberá Villanueva: Ana.Barbera@alu.uclm.es
* Ramón Ángel Aguilar Rodríguez : RamonAngel.Aguilar@alu.uclm.es

## Estaciones de Servicio y Energía: Encontrando la Configuración Óptima

## 1. Introducción

¡Noticias emocionantes! El **Ministerio de Transporte y Movilidad Sostenible** ha quedado muy impresionado con las soluciones desarrolladas en nuestro primer trabajo, la Práctica 1. Están particularmente interesados en implementar estos algoritmos en la planificación de rutas de vehículos autónomos, con A* como el método MÁS efectivo para identificar el camino óptimo de manera eficiente. Para avanzar en este proyecto, el Ministerio tiene como objetivo establecer estratégicamente estaciones de servicio en áreas urbanas para apoyar su flota de vehículos autónomos. Estas estaciones funcionarán como centros de flota y proporcionarán servicios esenciales para los vehículos.

Para lograr esto, han solicitado **nuestra experiencia técnica para determinar la distribución óptima de estas estaciones** en los mapas de la ciudad. Sin embargo, no todas las intersecciones son elegibles como ubicación de una estación; el Ministerio ha preseleccionado intersecciones candidatas basándose en criterios específicos establecidos por sus equipos técnicos y administrativos. Su principal enfoque es la sostenibilidad y el acceso equitativo, con el objetivo de garantizar que todos los ciudadanos estén razonablemente cerca de una estación de servicio. Entre estos puntos seleccionados, solo se elegirá un número específico. Para facilitar nuestra tarea de determinar cuáles deben ser, han proporcionado datos sobre la cobertura poblacional de cada intersección candidata, lo que nos permite tener en cuenta tanto el acceso como la cobertura en nuestra estrategia de distribución.

El objetivo principal es garantizar un acceso eficiente a la máxima población posible, manteniendo una distribución equilibrada en toda la ciudad, una consideración vital para un sistema de transporte completamente autónomo.

### 1.1 Objetivos del Laboratorio

En esta práctica, aplicaremos técnicas de búsqueda metaheurística para resolver problemas de optimización combinatoria.

El primer objetivo es comprender la tarea y formularla desde la perspectiva de la búsqueda metaheurística. Implementaremos al menos dos algoritmos:

* **Búsqueda Aleatoria**, como un punto de partida básico que generará múltiples soluciones, evaluará cada una y devolverá la mejor.

* **Algoritmo Genético**, que permitirá configurar varios parámetros, como el tamaño de la población, la tasa de mutación y el número de generaciones, entre otros.


Además, para la evaluación no-continua se tendrá que implementar la Ascensión de Cplinas o Hill Cilimbing (obligatoriamente), y opcionalmente implementar el algoritmo **Iterated Local Search** (ILS), ambos explicados en el Tema 7.

A continuación, analizaremos y compararemos el rendimiento de estos algoritmos ejecutándolos en instancias de problemas de diferente complejidad.

Esperamos que esta práctica te ayude a profundizar en tu comprensión de las técnicas metaheurísticas y te anime a considerar cómo se pueden aplicar en problemas reales de optimización combinatoria.

**¡Buena suerte!**

## 2. Descripción del Problema

### 2.1 Problemas de Entrada

Cada escenario se proporcionará en un archivo en formato `json` que contiene la siguiente información, con el formato de un diccionario cuyas claves son:

* `address`: La dirección utilizada.
* `distance`: Radio máximo utilizado para definir intersecciones y segmentos alrededor de la dirección.
* `intersections`: Una lista de diccionarios con información sobre las intersecciones.
* `segments`: Una lista de diccionarios con información sobre los segmentos, que representan las calles entre dos intersecciones.
* `candidates`: Una lista de pares (identificador, población) que contiene las intersecciones candidatas. Notad que los identificadores en esta lista deben estar incluidos en la lista de intersecciones.
* `number_stations`: El número de estaciones de servicio que se deben ubicar, que no debe superar el número de candidatos.

Cada diccionario en `intersections` incluye tres claves:

* `identifier`: Identificador de la intersección
* `longitude`: Longitud de la intersección
* `latitude`: Latitud de la intersección

Cada diccionario en `segments` incluye cuatro claves:

* `origin`: Intersección de origen
* `destination`: Intersección de destino
* `distance`: Distancia entre las dos intersecciones
* `speed`: Velocidad máxima permitida entre las dos intersecciones

**IMPORTANTE**: `initial` y `final` ya no están incluidos en el archivo JSON, ya que no son necesarios. Durante la evaluación de una posible configuración, estos puntos iniciales y finales cambiarán varias veces. Esto puede requerir algunos ajustes en el código de la Práctica 1 para ejecutar A*. Estos cambios deben estar claramente indicados (tu código debe coincidir con el de la Práctica 1, excepto por estos cambios) y discutidos en la memoria de prácticas.

### 2.2. Ejemplo ilustrativo

Un posible ejemplo de este problema podría ser el que se muestra en la siguiente imagen, que representa una parte de la ciudad de Albacete:

![title](sample-problems-lab2/toy/example.png)

En este caso, el número de estaciones de servicio de vehículos que se deben ubicar es 4, entre las 15 intersecciones candidatas representadas con puntos azules (etiquetadas con la población cubierta). Una posible solución se representa con puntos verdes.

---

##### Nota:

* El archivo que contiene la imagen debe guardarse en la ruta indicada en el código de esta celda.

---

### 2.3 Definición formal del problema

Necesitamos elegir $s$ estaciones de entre $c$ intersecciones candidatas o elegibles, con $s<c$. Por lo tanto, nuestro objetivo es decidir en cuál de estas $c$ intersecciones candidatas debemos ubicar las $s$ estaciones de servicio de vehículos, de manera que se minimice el tiempo promedio de viaje que cada habitante tarda desde su hogar hasta la estación más cercana. Si denotamos por $S$ al vector de tamaño $s$ que contiene las intersecciones en las que se ubican las estaciones de vehículos y por $C$ al vector de intersecciones candidatas que contiene el par (id, pop) para cada intersección candidata, entonces formalmente, queremos resolver el siguiente problema de optimización:

$$
S^* = \arg\min_{S} \frac{1}{\sum_{i=0}^{c-1} C[i].pop} \cdot \min_{j=0,\dots,s-1} \left\{\sum_{i=0}^{c-1} \; C[i].pop \cdot time(C[i].id,S[j])\right\}
$$

donde:
- $C[i].pop$ representa la población (número de habitantes) cubierta por la intersección candidata $i$.
- $C[i].id$ es el identificador de la intersección candidata $i$.
- $time(A,B)$ representa el menor tiempo real para viajar desde la intersección $A$ hasta la intersección $B$.

Las siguientes consideraciones deben tenerse en cuenta respecto a la expresión anterior:
- Estamos tratando con un problema de minimización.
- La cardinalidad del espacio de búsqueda es:

$$
\binom{c}{s} = \frac{c!}{(c-s)!s!}
$$

por ejemplo, si tenemos 20 intersecciones elegibles y 4 estaciones de vehículos, el número de soluciones posibles es 210, no demasiadas; pero si tenemos 100 candidatos y 10 estaciones, entonces el número de soluciones posibles es $1.7\times10^{13}$ ($5.3\times 10^{20}$ con 20 estaciones).

## 3. Desarrollo de la práctica

Antes de implementar los algoritmos, primero debes considerar definir los elementos básicos en este tipo de problemas, a saber:

- Una representación conveniente para las soluciones (configuraciones, cromosomas, individuos, ...) del problema que se utilizarán en los algoritmos de optimización combinatoria. Piensa detenidamente en las distintas opciones y tendrás que discutirlas en el informe de la tarea.

- Implementar un mecanismo de evaluación para gestionar las evaluaciones realizadas por los algoritmos de optimización combinatoria. A continuación, se detallará cómo debe realizarse la evaluación.

- Notas importantes:
    - En el caso de que A* no devuelva ninguna solución (coste = inf), reemplazad este valor por un número muy alto en comparación con el tiempo máximo en nuestro problema. Reflexiona sobre la necesidad de esto y discútelo en el informe.
    - Podéis aprovechar el mecanismo de evaluación para guardar algunos cálculos, recopilar estadísticas e imprimir los resultados.
    - Tened en cuenta que esta tarea requiere que ya hayas resuelto la Práctica 1, y necesitarás reutilizar el código implementado para resolver esta práctica.
   
Tendréis que resolver muchos problemas similares a los de la Práctica 1. Los mapas serán los mismos, pero los problemas necesitan incorporar nueva información, que es la lista de intersecciones candidatas y, para cada una de ellas, la población que cubren. El número de estaciones que se deben ubicar también se indica en el problema como `number_stations`.


### 3.1 Evaluación de una solución

Dada una instancia específica del problema a resolver, y asumiendo que $C$ denota su lista de intersecciones candidatas, el valor de cada posible solución $S$ debe calcularse como:

$$value(S) = \frac{1}{\sum_{i=0}^{c-1} C[i].pop} \cdot \min_{j=0,\dots,s-1} \left\{\sum_{i=0}^{c-1} \; C[i].pop \cdot time(C[i].id,S[j])\right\}$$

de acuerdo con la fórmula presentada en la sección 2.2.

## 4. Plan de trabajo

### 4.1. Tareas

* Transferid y adaptad vuestro código de la Práctica 1 para resolver búsquedas con A* que necesitaréis aquí:
    * Reutilizad la mayor parte del código necesario de vuestra Práctica 1.
    * Describid qué se ha modificado, por qué y cómo.

* Procesad los nuevos archivos JSON y guardad el problema de acuerdo con lo siguiente:
    * Además de las clases de búsqueda (Problem_2, State, Action, ...), deberéis trabajar con las intersecciones candidatas.
    * Construid un mecanismo capaz de almacenar y recuperar las intersecciones candidatas y la población asociada a cada una.

* Representación de una posible configuración:
    * Entre las vistas en el Tema 6, encontrad la representación más adecuada para este problema y adaptadla, teniendo en cuenta que cada problema tiene valores distintos para el número total de intersecciones, intersecciones elegibles y el número solicitado de estaciones.
    * Conectadla a un método adecuado para evaluar cada una considerando las indicaciones mencionadas anteriormente en el punto 3.2.

* Implementación de algoritmos:
    * Implementad, al menos, los dos algoritmos requeridos (Búsqueda Aleatoria y un Algoritmo Genético — GA). Tened en cuenta que para la evaluación no continua, también deberéis agregar Hill Climbing y, opcionalmente, ILS.
    * En el GA, aseguraos de haber implementado la generación de una población junto con los elementos principales dentro del bucle principal: selección, cruce, mutación y combinación de generaciones.

* Experimentación y análisis:
    * Los parámetros que se puedan ajustar deben ser explorados adecuadamente, también en relación con los problemas dados (dimensionalidad, complejidad, etc.).
    * También deberéis estudiar el rendimiento resultante en términos de desempeño, convergencia, número de generaciones, etc.
    * Comparad la Búsqueda Aleatoria y el Algoritmo Genético, asegurándoos de obtener resultados consistentes.

* Informe:
    * Redactad un informe detallando el proceso seguido, las estrategias implementadas y los resultados obtenidos, junto con gráficos y comparaciones visuales.


### 4.2. Evaluación de la práctica

En la modalidad de **evaluación continua**, la evaluación de la práctica se realizará a través de un examen individual en el que se tendrá en cuenta lo siguiente:

* Definición e implementación correcta de la representación de la configuración y de la función de evaluación: 25%
* Implementación correcta del algoritmo genético: 50%, que cubre
    * El bucle general para las generaciones es correcto: 10%
    * Los distintos operadores están correctamente diseñados y codificados: 40%
* Eficiencia y optimización: 15%
* Experimentación realizada y análisis de resultados: 10%

Es necesario que la Práctica 1 esté correctamente integrada y que la Búsqueda Aleatoria funcione de manera consistente para que todos los estudiantes puedan utilizarla como un punto de partida base adecuado.

Todo esto se ponderará según el nivel de conocimiento que el estudiante demuestre sobre la práctica en caso de que el examen sea una entrevista personal.

En la modalidad de **evaluación no continua**, la evaluación se modificará como se indica a continuación:

* Definición e implementación correcta de la representación de la configuración y de la función de evaluación: 15%
* Implementación correcta del algoritmo genético: 40%, que cubre
    * El bucle general para las generaciones es correcto: 7%
    * Los distintos operadores están correctamente diseñados y codificados: 33%
* Implementación correcta del algoritmo de Hill Climbing (obligatorio): 15%
* Implementación correcta del algoritmo ILS (opcional): 5%
* Eficiencia y optimización: 15%
* Experimentación realizada y análisis de resultados: 10%


### 4.3. Fechas importantes

* Fecha límite para entregar el código: **13 de diciembre de 2024**.
* Fecha límite para la entrega del informe: **Final del semestre**.



In [69]:
import json
from abc import ABC, abstractmethod
import heapq #priority queue
import time
import random
import logging #Imprimir archivo
EstadoDestino = None

In [70]:
class Estado:
    def __init__(self, interID, latitud, longitud):
        self.intersec = interID
        self.latitud = latitud
        self.longitud = longitud

    def __eq__(self, other): #equals
        return self.intersec == other.intersec

    def __lt__(self, other):  # less than
        return self.intersec < other.intersec

    def __hash__(self): #Identifica un objeto
        return hash(self.intersec)
    
    def __str__(self): #Imprimir el estado de cada nodo por pantalla
        return "Este es el estado " + str(self.intersec)

In [71]:
class Gproblema:
    def __init__(self, djson):
        #Apertura de json
        with open(djson, 'r') as archivo:
            self.datos = json.load(archivo)
        if self.datos is None: 
            raise ValueError("Error cargando el JSON")

        #Generación de estadísticas
        self.estadisticas = {
            "nodos_generados": 0,
            "expandidos": 0,
            "coste": 0,
            "profundidad": 0
        }

    #Es más rápido así porque cada vez que quisieramos buscar en el json tendríamos que leer todo lo anterior hasta encontrar
    #la intersección buscada. Así, lo que conseguimos es que al ser clave el identificador, es casi instantánea la búsqueda
    def Crear_Estructura(self):
        self.interseciones = {}
        self.vmax = 0
        self.candidatos = {}
        #Generación de diccionario de intersecciones
        #intersections en el json es un vector de diccionarios que contienen un id, la latitud y la longitud
        for interseccion in self.datos["intersections"]: #interseccion es cada uno de los diccionarios
            #metemos en el diccionario de intersecciones, donde la clave es el identificador, una tupla con la longitud y latitud
            self.interseciones[interseccion["identifier"]] = (interseccion["latitude"], interseccion["longitude"])
        self.tpop = 0;
        
        #Cambios
        indice = 0  # Contador
        self.iDCandidates = {}
        for a in self.datos["candidates"]:    
            self.candidatos[a[0]] = a[1]
            self.tpop = self.tpop + a[1]
            self.iDCandidates[indice] = a[0]  # Usamos el contador como clave
            indice += 1

        self.segmentos = {}
        #Generación de diccionario de segmentos que guardan los nodos destino
        #Cada posible origen se mira una única vez, por que en segmentos se repite muchas veces.
        for segmento in self.datos["segments"]:
                #Cambiamos priority queue por una lista y ordenarla cuando terminemos 
            if(segmento["origin"] not in self.segmentos):
                self.segmentos[segmento["origin"]] = [(segmento["destination"], segmento["distance"], segmento["speed"])]
            else: 
                self.segmentos[segmento["origin"]].append((segmento["destination"], segmento["distance"], segmento["speed"]))
            if segmento["speed"] > self.vmax:
                self.vmax = segmento["speed"]
        for i in self.segmentos:
            self.segmentos[i].sort()
            #IMPLEMENTAR: ORDENAR LAS LISTAS
        #Generación de estado inicial y final
        #Cambio, quitamos el estado inicial y final de problema
        #longi = 0
        #lat = 0
        #lat, longi = self.interseciones[self.datos["initial"]]
        #self.estadoini = Estado(self.datos["initial"], lat, longi)
        #lat, longi = self.interseciones[self.datos["final"]]
        #self.estadofin = Estado(self.datos["final"], lat, longi)
        #Cambios
    def setIniFin(self, inicial, final):
        lat, longi = self.interseciones[inicial]
        self.estadoini = Estado(inicial, lat, longi)
        lat, longi = self.interseciones[final]
        self.estadofin = Estado(final, lat, longi)

In [72]:
class Accion:
    def __init__(self, idOg, idDest, speed, distancia):
        self.idOg = idOg
        self.idDest = idDest
        self.speed = speed
        self.distancia = distancia
        self.tiempo = self.distancia / self.speed

In [73]:
class Nodo:
    def __init__(self, estado, padre, accion):
        self.estado = estado
        self.padre = padre
        self.accion = accion
        if padre:
            self.profundidad = padre.profundidad + 1
            self.costoG = padre.costoG + (accion.distancia/((accion.speed*1000)/3600))
        else:
            self.profundidad = 0
            self.costoG = 0 #Coste de la accion
            
    
    #Utilización de Manhattan para la heurística
    def heuristica(self,EstadoDestino, velMax):
        return  (abs(self.estado.latitud - EstadoDestino.latitud) + abs(self.estado.longitud - EstadoDestino.longitud))/((velMax*1000)/3600)
        

    def caminos_sol (self):
        camino = []
        nodo_actual = self 
        tiempoTotal = 0
        distanciaTotal = 0
        while nodo_actual:
            camino.append(nodo_actual.estado.intersec)
            distanciaTotal += nodo_actual.accion.distancia if nodo_actual.accion else 0
            tiempoTotal += (nodo_actual.accion.distancia/(nodo_actual.accion.speed*1000)) if nodo_actual.accion else 0
            nodo_actual = nodo_actual.padre
        camino.reverse() #Para que el camino aparezca en orden
        return camino, f" Se recorrio {distanciaTotal} metros en {tiempoTotal*60} minutos" # se multiplica por 60 para pasar de horas a minutos
    
    def __lt__(self, other):
        # Definimos que un Nodo es "menor que otro" si su costo es menor
        return self.costoG < other.costoG

In [74]:
class Busqueda(ABC): #ABC sirve para que la clase pueda tener métodos abstractos
    #Practica2: Añadimos inicial y final como parametros
    def __init__(self, problema,inicial,final):
        self.problema = problema
        problema.setIniFin(inicial,final)

    def Buscar(self):
        #iniciamos cronómetro para obtener el tiempo total de ejecución del programa
        inicio = time.time()
        #Creamos frontera, cerrados e insertamos el nodo inicial
        self.problema.Crear_Estructura()
        fronteraa = []
        cerrados = set() #Conjunto que no admite duplicados
        #Insertar es un método abstracto que se implementa en cada método de búsqueda, sirve para meter el nodo inicial
        self.Insertar(Nodo(self.problema.estadoini, None, None), fronteraa) 
        solucion = []
        #Comenzamos el bucle de búsqueda
        while True: #Es infinito por que ya existen condiciones que lo paran. Y no sabemos los nodos que hay que buscar
            if self.Vacia(fronteraa):
                print("No hay solucion")
                break
            nodo = self.BorrarPrimero(fronteraa)
            if self.TestObjetivo(nodo):
                solucion = nodo.caminos_sol()
                break
            if nodo.estado not in cerrados:
                cerrados.add(nodo.estado)
                self.InsertarTodo(self.Expandir(nodo),fronteraa)
        #Finalizamos cronómetro
        fin = time.time()

        #Creacioón de txt con la información
        logging.basicConfig(filename="Resultadotxt.txt", level=logging.INFO)
        logging.info("Problema:" + str(self.problema.datos["address"]))
        logging.info("Nodos Generados: " + str(self.problema.estadisticas["nodos_generados"]))
        logging.info("Nodos Expandidos: " + str(self.problema.estadisticas["expandidos"]))
        logging.info("Tiempo de ejecucion: " + str(fin - inicio))
        logging.info("Longitud de la solución: " + str(nodo.profundidad))
        logging.info("Coste de la solución: " + str(nodo.costoG))
        logging.info("Solución: " + str(solucion))
        logging.info("\n")
        logging.info("-----------------------------------------------------------")
        logging.info("\n")

        #Imprimimos la información
        print("Resultados también descargados en un archivo txt llamado Resultadotxt.txt")
        print("Problema:" + str(self.problema.datos["address"]))
        print("Nodos Generados: " + str(self.problema.estadisticas["nodos_generados"]))
        print("Nodos Expandidos: " + str(self.problema.estadisticas["expandidos"]))
        print("Tiempo de ejecucion: " + str(fin - inicio))
        print("Longitud de la solución: " + str(nodo.profundidad))
        print("Coste de la solución: " + str(nodo.costoG))
        print("Solución: " + str(solucion))
        return nodo.costoG

    #Comprobamos si es el estado final
    def TestObjetivo(self,nodo): #Porque nodo le vamos a hacer el mismo tratamiento dando igual el tipo de algoritmo
        if nodo.estado == self.problema.estadofin:
            return True
        return False
    
    def Solucion(self,nodo):
        nodo.caminossol()
    
    #Sacamos los hijos del nodo que pasamos por parámetro
    def Expandir(self,nodo):
        self.problema.estadisticas["expandidos"] += 1

        if nodo.estado.intersec in self.problema.segmentos:
            destinos = self.problema.segmentos[nodo.estado.intersec]
        else:
            print(f"Clave no encontrada: {nodo.estado.intersec}")
            return []  # No expandimos este nodo #Devuelve una priority queue que esta ordenada con todos los destinos
        nodosl = []
        nodoid = 0
        nodod = 0
        nodos = 0
        lat = 0
        longi = 0
        for destino in destinos:
            (nodoid, nodod, nodos) = destino #Sacamos el que tiene la prioridad, y devuelve la tupla de distancia y speed
            lat,longi = self.problema.interseciones[nodoid]
            #añadimos un nodo, con su padre y su accion
            nodosl.append(Nodo(Estado(nodoid,lat,longi), nodo,Accion(nodo.estado.intersec,nodoid,nodos,nodod)))
        return nodosl

            

    #En las búsquedas no informadas siempre vamos a sacar el primero de una lista 
    #En búsqueda informada siempre vamos a buscar por una heurística y para eso utilizamos priority queue        
    #Función que insertara en frontera el nodo inicial
    @abstractmethod
    def Insertar(self, nodo, frontera):
        pass

    #Función que comprobará si es vacía
    @abstractmethod
    def Vacia(self, frontera):
        pass

    #Función que borrará el primer nodo de la frontera
    @abstractmethod
    def BorrarPrimero(self, frontera):
        pass
    
    #Función que inserta una lista de nodos
    @abstractmethod
    def InsertarTodo(self, nodos,frontera):
        pass

In [75]:
class AEstrella (Busqueda):
    def Insertar(self, nodo, frontera):
        self.problema.estadisticas["nodos_generados"] +=1
        h = nodo.heuristica(self.problema.estadofin,self.problema.vmax)
        heapq.heappush(frontera,(nodo.costoG + h, nodo))
        
    def Vacia(self, frontera):
        if not frontera:
            return True
        return False
        
    def BorrarPrimero(self, frontera):
        p, n = heapq.heappop(frontera)
        return n
    
    def InsertarTodo(self, nodos,frontera):
        for nodo in nodos:
            self.problema.estadisticas["nodos_generados"] +=1
            heapq.heappush(frontera, (nodo.costoG + nodo.heuristica(self.problema.estadofin,self.problema.vmax), nodo))
        

In [76]:
class BusquedaMA(ABC):
    def __init__(self,djson):
        self.problema=Gproblema(djson)
        self.problema.Crear_Estructura()
        self.psolucion=[]
        self.calculados= {}

    def EsSolucion(self, lpsol):
        sumat = 0
        for candidato in self.problema.candidatos:
            min_tiempo = float('inf')
            for psol in lpsol:
                par = (candidato, psol)
                if par in self.calculados:
                    tiempo = self.calculados[par]
                else:
                    buscador = AEstrella(self.problema, candidato, psol)
                    tiempo = buscador.Buscar()
                    self.calculados[par] = tiempo
                if tiempo < min_tiempo:
                    min_tiempo = tiempo
            # Para el candidato, sumas su peso por el tiempo mínimo a una estación
            sumat += self.problema.candidatos[candidato] * min_tiempo
        sol = sumat / self.problema.tpop
        return sol
        

     
                 

## Búsqueda aleatoria


In [77]:
class BusquedaAleatoria(BusquedaMA):

    def Bus(self,sem,n):
        #iDCandidates
        inicio = time.time()
        random.seed(sem)

        long = len(self.problema.datos["candidates"])
        clavesAleatorias =  random.sample(range(long), self.problema.datos["number_stations"])
        clavesdic = {}
        for i in clavesAleatorias:
            clavesdic[i] = 0
        vIds = [0,""] * long
        sIds = []
        for i in range(long):
            if i in clavesdic:
                vIds[i] = [1,self.problema.iDCandidates[i]]
                sIds.append(self.problema.iDCandidates[i])
                continue
            vIds[i] = [0,self.problema.iDCandidates[i]]
        valor = self.EsSolucion(sIds)

        for i in range(n):
            vecinos = []
            mvecino = []
            mvalorvec = 3600 * 6
            vecsIds = sIds
            mvecsIds = []
            for i in range(long):
                for i in range(long):
                    for j in range(long):
                        vecino = vIds
                        if(i==j or vecino[i][0] == vecino[j][0]):
                            continue
                        primer = vecino[i][0]
                        vecino[i][0] = vecino[j][0]
                        vecino[j][0] = primer
                        for k in range(len(vecsIds)):
                            if(vecsIds[k]==vecino[i][1] and vecino[i][0] == 0):
                                vecsIds.pop(k)
                                vecsIds.append(vecino[j][1])
                                break
                            elif(vecsIds[k]==vecino[j][1] and vecino[j][0] == 0):
                                vecsIds.pop(k)
                                vecsIds.append(vecino[i][1])
                                break
                        valorvec = self.EsSolucion(vecsIds)
                        if(valorvec<mvalorvec):
                            mvalorvec = valorvec
                            mvecino = vecino
                            mvecsIds = vecsIds
            if(valor>mvalorvec):
                valor = mvalorvec
                vIds = mvecino
                sIds = mvecsIds
            else:
                clavesAleatorias =  random.sample(range(long), self.problema.datos["number_stations"])
                clavesdic = {}
                for i in clavesAleatorias:
                    clavesdic[i] = 0
                vIds = []
                sIds = []
                for i in range(long):
                    if i in clavesdic:
                        vIds[i] = [1,self.problema.iDCandidates[i]]
                        sIds.append(self.problema.iDCandidates[i])
                        continue
                    vIds[i] = [0,self.problema.iDCandidates[i]]
                valor = self.EsSolucion(sIds)
        
        fin = time.time()
        print(f"Solucion más optima encontrada {sIds}" )
        print(f"Fitness {valor}" )
        print(f"Número de iteraciones{n}")
        print(f"Tiempo de ejecución {(fin-inicio)//60} minutos {(fin-inicio)%60} segundos")




#tengo que coger de essolucion el valor optimo 

 ProcedimientoAlgoritmoGenético
 1.-t=0
 2.- inicializarP(t)
 3.-evaluarP(t)
 4.-Mientras(nosecumplalacondicióndeparada)hacer
 4.1.-t=t+1
 4.2.-seleccionarP’(t)desdeP(t-1)
 4.3.-recombinarP’(t)
 4.4.-mutaciónP’(t)
 4.5.-evaluarP’(t)
 4.6.-P(t)=combinar(P’(t),P(t-1))

## Algoritmo Genético


In [78]:
class AlgoritmoGenetico(BusquedaMA):
    def BusA(self,sem):
        random.seed(sem)

        t = 0
        #Generar población inicial
        Pt = self.InicioPoblacion(sem)
        #Calcular la evaluacion de cada individuo
        evaluados = self.EvaluarP(Pt)
        aptitudes = evaluados.copy()  # Guardamos las aptitudes para combinarlas más tarde
        while not t >= 500 : 
            t += 1
            # Seleccionar P'(t) desde P(t-1): Selección de padres
            Pprima = self.Selec(Pt, evaluados)
            # Recombinación P'(t): Cruzar los padres seleccionados
            Pprima = self.Recombinar(Pprima)
            #  Mutación P'(t): Aplicar mutaciones aleatorias
            Pprima = self.Mutar(Pprima)
            # 4.5.- Evaluar P'(t)
            evaluados_prima = self.EvaluarP(Pprima)
            # 4.6.- P(t) = combinar(P'(t), P(t-1))
            Pt, aptitudes = self.CombinarP(Pt, aptitudes, Pprima, evaluados_prima)

        #devolucion de la mejor solución encontrada
        mejorSol = self.ElMejor(Pt, aptitudes)
        return mejorSol
    
    def InicioPoblacion(self, sem):
        random.seed(sem)
        poblacion = []
        
        # Extraer solo el primer valor de cada subvector en "candidates"
        candidatos = [c[0] for c in self.problema.datos["candidates"]]
        nCandidatos = len(candidatos)  # Cantidad de candidatos
        
        for _ in range(nCandidatos):  # Tamaño de la población igual al número de candidatos
            # Elegir aleatoriamente los valores de los primeros elementos
            individuo = random.sample(candidatos, self.problema.datos["number_stations"])
            poblacion.append(individuo)
    
        return poblacion

    
    def EvaluarP (self, poblacion):
        return [self.EsSolucion(individuo) for individuo in poblacion]
    

    #Selección proporcional al fitness
    def Selec(self, poblacion, evaluados):
        losEvaluados = sum(evaluados)
        if losEvaluados == 0:
            print("Advertencia: Todas las evaluaciones son cero. Selección aleatoria aplicada.")
            return random.choices(poblacion, k=len(poblacion))
        prob = [evaluado / losEvaluados for evaluado in evaluados]
        padres = random.choices(poblacion, weights=prob, k=len(poblacion))
        return padres

    

    def Recombinar(self, poblacion):
        nGeneracion = []
        for i in range(0, len(poblacion), 2):
            padre1 = poblacion[i]
            padre2 = poblacion[i + 1] if i + 1 < len(poblacion) else poblacion[0]

            # Elegir dos puntos de cruce aleatorios
            i, d = sorted(random.sample(range(len(padre1)), 2))
            
            # Inicializar los hijos con segmentos externos de los padres
            #Cogemos el primer segmento de la izquierda del padre 1
            #En el medio ponemos el trozo de en medio del padre dos,
            #Se suma 1 para que se coja i:d , porque si no se cogeria de i a d-1
            #y por último se añade el resto del padre 1 y se le suma 1 para que llegue al final

                #Cruce básico
            hijo1 = padre1[:i] + padre2[i:d+1] + padre1[d+1:] #Con los dos puntos se consigue que pare en el punto de corte d
            hijo2 = padre2[:i] + padre1[i:d+1] + padre2[d+1:]
            
            nGeneracion.extend([hijo1, hijo2])
        return nGeneracion

    
    def Mutar(self, poblacion):
        for individuo in poblacion: #El individuo es una solucion candidata representada como una lista de genes
            if random.random() < 0.05: #Genera entre 0 y 1, si es menor a 0.2 muta, es decir, el 20% de la poblacion muta
                idx1, idx2 = random.sample(range(len(individuo)), 2) #Coge de manera aleatoria dos genes y los muta
                individuo[idx1], individuo[idx2] = individuo[idx2], individuo[idx1] #intercambiamos índices
        return poblacion
    

    
    def CombinarP(self, Pt, evaluados, Pprima, evPrima):
        combinados = list(zip(Pt + Pprima, evaluados + evPrima))
        combinados.sort(key=lambda x: x[1], reverse=True)
        nueva_poblacion = [ind for ind, _ in combinados[:len(Pt)]]
        nEvaluacion = [eval for _, eval in combinados[:len(Pt)]]
        return nueva_poblacion, nEvaluacion

    def ElMejor(self, poblacion, evaluados):
        max_evaluacion = max(evaluados)
        mejor_individuo = poblacion[evaluados.index(max_evaluacion)]
        return mejor_individuo
        


# Algoritmos de no continua

### Hill-Climbing

In [79]:
class hill_climbing(BusquedaMA):

    def Bus(self, sem, it):
        """
        Método de búsqueda por Hill Climbing.
        param sem: Semilla para la generación aleatoria.
        param it: Número de iteraciones a realizar.
        return: Tupla con la mejor solución encontrada y su valor.
        """
        random.seed(sem)

        # Generar solución inicial aleatoria
        sol = self.Gen_sol_random()
        #comprobar si es solución
        actual = self.EsSolucion(sol)

        for _ in range(it):
            #generar vecinos
            vecinos = self.gen_vecinos(sol)
            
            #elegimos al mejor vecino
            mejor_vecino = None
            mejor_valor = actual

            for vecino in vecinos:
                v = self.EsSolucion(vecino)
                #si el vecino es mejor que la solución actual
                if v < mejor_valor:
                    mejor_valor = v
                    mejor_vecino = vecino
            
            #si no hay mejor vecino, salimos
            if mejor_vecino is None:
                break
            
            #si hay mejor vecino, lo actualizamos
            sol = mejor_vecino
            actual = mejor_valor

        return sol, actual


            

    def Gen_sol_random(self):
        """
        Método para generar una solución aleatoria.
        return: Lista de identificadores de intersecciones representando una solución aleatoria.
        """
        candidatos =[ c[0] for c in self.problema.datos["candidates"]] #Extraemos los identificadores de las intersecciones candidatas
        return random.sample(candidatos, self.problema.datos["number_stations"]) #devolvemos una lista de identificadores de intersecciones aleatorios, con el número de estaciones 

    def gen_vecinos(self, sol):
        """
        Método para generar vecinos de una solución dada.
        param sol: Lista de identificadores de intersecciones representando una solución.
        return: Lista de vecinos generados a partir de la solución dada.
        """
        vecinos = []
        for i in range(len(sol)):
            for j in range(i + 1, len(sol)): # para evitar repetir combinaciones
                # Intercambiar dos intersecciones para generar un vecino
                vecino = sol[:] #hacemos una copia de la solución actual
                vecino[i], vecino[j] = vecino[j], vecino[i] #cambiamos los valores de las posiciones i y j para generar un vecino
                
                vecinos.append(vecino) #añadimos el vecino a la lista de vecinos
        return vecinos
        

# MAIN
llamadas algoritmo


In [80]:
BusquedaAL = BusquedaAleatoria(r"C:\Users\ramon\Desktop\Archivos uni\4 año\Inteligentes\Practica2\calle_agustina_aroca_albacete_250_0_candidates_75_ns_7.json")

BusquedaAL.Bus(69,10)

# BusquedaAL = AlgoritmoGenetico(r"C:\Users\Ana Barberá\Desktop\Práctica 2 Algoritmos metaheurísicos-20241215\sample-problems-lab2\medium\calle_agustina_aroca_albacete_500_1_candidates_89_ns_22.json")
# BusquedaAL.BusA(69)

# BusquedaMA = hill_climbing(r"C:\Users\Ana Barberá\Desktop\Práctica 2 Algoritmos metaheurísicos-20241215\sample-problems-lab2\medium\calle_agustina_aroca_albacete_500_1_candidates_89_ns_22.json")
# BusquedaMA.Bus(69, 1000) 

Clave no encontrada: 1557262702
Clave no encontrada: 1835353347
Resultados también descargados en un archivo txt llamado Resultadotxt.txt
Problema:Calle Agustina Aroca, Albacete
Nodos Generados: 32
Nodos Expandidos: 23
Tiempo de ejecucion: 0.0010008811950683594
Longitud de la solución: 11
Coste de la solución: 57.96935999999999
Solución: ([1572435237, 1572435249, 1574598070, 1572435255, 1554807113, 1526105986, 618009048, 1530209368, 1526105983, 1530209339, 335720312, 335607957], ' Se recorrio 483.078 metros en 0.9661560000000001 minutos')
Clave no encontrada: 1557262702
Clave no encontrada: 1835353347
Clave no encontrada: 1560897811
Resultados también descargados en un archivo txt llamado Resultadotxt.txt
Problema:Calle Agustina Aroca, Albacete
Nodos Generados: 110
Nodos Expandidos: 74
Tiempo de ejecucion: 0.0
Longitud de la solución: 19
Coste de la solución: 91.24780000000001
Solución: ([1572435237, 1572435249, 1574598070, 1572435255, 1554807113, 1526105986, 618009048, 1530209368, 152

IndexError: list assignment index out of range